In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent
from agential.cog.prompts.reflexion import REFLEXION_COT_REFLECT_FEWSHOT_GSM8K_EXAMPLES

In [2]:
from langchain_community.chat_models.openai import ChatOpenAI

In [3]:
llm_action = ChatOpenAI(api_key="")
llm_reflect = ChatOpenAI(api_key="")

In [13]:
testAgent = ReflexionCoTAgent(action_llm= llm_action, self_reflect_llm=llm_reflect)

In [4]:
key = "624"
context = None

In [15]:
testAgent.generate(question = "James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?",
                   key = key ,
                   context= context, examples= REFLEXION_COT_REFLECT_FEWSHOT_EXAMPLES_GSM8K_NO_CONTEXT)

S
S


[(True,
  '624',
  ('Thought: James writes a total of 3 x 2 x 2 = <<3*2*2=12>>12 pages per week.In a year, he writes 12 x 52 = <<12*52=624>>624 pages.Action: Finish[624]',
   'Action: Finish[624]',
   'Observation: Answer is CORRECT'))]

In [5]:
from agential.cog.agent.reflexion import ReflexionCoTAgent

In [6]:
TestAgent1 = ReflexionCoTAgent(self_reflect_llm= llm_reflect , action_llm= llm_action)

In [7]:
TestAgent1.generate(question= "James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?", key = key, examples=REFLEXION_COT_REFLECT_FEWSHOT_GSM8K_EXAMPLES)

S


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


S


[(True,
  '624',
  ('Thought: James writes 3 pages x 2 friends x 2 times per week = 12 pages per week.In a year, there are 52 weeks, so James writes 12 pages x 52 weeks = <<3*2*2*52=624>>624 pages a year.Action: Finish[624]',
   'Action: Finish[624]',
   'Observation: Answer is CORRECT'))]